## Apply Quantum SVM with PCA + PQK (XYZ ansatz)

### 0. Load libraries

In [ ]:
import os
from os import listdir
import pandas as pd
from collections import Counter

import torch
import torch.nn as nn
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

import pennylane as qml
from pennylane import numpy as np

import data
import embedding

### 1. Load dataset

In [ ]:
feature_reduction = False
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('COVID19', feature_reduction, classes)

In [ ]:
print("X_train:",X_train.shape,"/X_test:",X_test.shape,"/Y_train:",Y_train.shape,"/Y_test:",Y_test.shape)

In [ ]:
print(Counter(Y_train), Counter(Y_test))

### 2. Apply PCA

In [ ]:
full_X = pd.concat([pd.DataFrame(X_train), pd.DataFrame(X_test)], axis=0)
print("Before PCA:",full_X.shape)

pca_XYZ = PCA(n_components=15)
fixed_X_XYZ = pca_XYZ.fit_transform(X=full_X)
print("After PCA:",fixed_X_XYZ.shape)

fixed_X_XYZ_train = fixed_X_XYZ[:98,:]
fixed_X_XYZ_test = fixed_X_XYZ[98:,:]

X_XYZ_train=[]
X_XYZ_test=[]
for x in fixed_X_XYZ_train:
    X_XYZ_train.append(x / 2)
for x in fixed_X_XYZ_test:
    X_XYZ_test.append(x / 2)

X_XYZ_train = np.array(X_XYZ_train)
X_XYZ_test = np.array(X_XYZ_test)

print("(XYZ) :", fixed_X_XYZ_train.shape, fixed_X_XYZ_test.shape)

### 3. Define PQK and embedding circuits for encoding methods

In [ ]:
def get_quantum_embedding(embedding_type):
    if embedding_type == "XYZ":
        def XYZ_embedding(x):
            embedding.repeat_XYZ_8_qubits(x)
        return XYZ_embedding

In [ ]:
# Projected Quantum Kernel
class ProjectedQuantumKernel(torch.nn.Module):
    def __init__(self, embedding_type, num_features, gamma_factor=1):
        super(ProjectedQuantumKernel, self).__init__()  
        self.embedding = get_quantum_embedding(embedding_type)
        self.gamma_factor = gamma_factor
        self.num_features = num_features
        self.circuit = None 

    def construct_circuit(self):
        dev = qml.device('default.qubit', wires=8)
        @qml.qnode(dev, interface='torch')
        def circuit(x):
            #print("input:",x.shape)
            x = self.embedding(x)
            return ([qml.expval(qml.PauliX(wires=i)) for i in range(8)]
                     + [qml.expval(qml.PauliY(wires=i)) for i in range(8)]
                     + [qml.expval(qml.PauliZ(wires=i)) for i in range(8)])

        self.circuit = circuit

    def forward(self, X1, X2=None):
        dim1 = len(X1)
        if X2 is not None:
            dim2 = len(X2)
        else:
            dim2 = dim1


        self.construct_circuit()
        
        valsX1 = torch.tensor([self.circuit(x1) for x1 in X1])  # Use torch tensor
        valsX1 = valsX1.view(dim1, 3, -1)  # Reshape to (dim1, 3, num_qubits)

        if X2 is not None:
            valsX2 = torch.tensor([self.circuit(x2) for x2 in X2])
            valsX2 = valsX2.view(dim2, 3, -1)
        else:
            valsX2 = valsX1

        valsX_X1 = valsX1[:, 0]
        valsX_X2 = valsX2[:, 0]
        valsY_X1 = valsX1[:, 1]
        valsY_X2 = valsX2[:, 1]
        valsZ_X1 = valsX1[:, 2]
        valsZ_X2 = valsX2[:, 2]

        # Concatenate and calculate default gamma
        all_vals_X1 = torch.cat((valsX_X1, valsY_X1, valsZ_X1), dim=1).view(-1)
        default_gamma = 1 / torch.var(all_vals_X1) / self.num_features

        # Initialize kernel matrix
        K = torch.zeros([dim1, dim2])

        # Compute the kernel matrix
        for i in range(dim1):
            for j in range(dim2):
                sumX = torch.sum((valsX_X1[i] - valsX_X2[j]) ** 2)
                sumY = torch.sum((valsY_X1[i] - valsY_X2[j]) ** 2)
                sumZ = torch.sum((valsZ_X1[i] - valsZ_X2[j]) ** 2)

                K[i, j] = torch.exp(
                    -default_gamma * self.gamma_factor * (sumX + sumY + sumZ)
                )

        return K

### 4. Define a kernel matrices from NQE 

In [ ]:
kernel_matrix_XYZ = ProjectedQuantumKernel(embedding_type="XYZ", num_features=15)

### 5. Apply SVM classifier

In [ ]:
svm_XYZ = SVC(kernel=kernel_matrix_XYZ, class_weight='balanced').fit(X_XYZ_train, Y_train)

### 6. Evaluate trained SVM with new ansatz by test dataset

#### 6-1) SVM with XYZ

In [ ]:
accs =[]
for i in range(5):
    predictions = svm_XYZ.predict(X_XYZ_test)

    print(predictions, Y_test)
    acc = balanced_accuracy_score(Y_test, predictions)
    accs.extend([acc])  

print("Accuracy with PCA and the kernel matrix of the XYZ encoding:", np.mean(accs),'±',np.std(accs))